In [4]:
## IMPORTING NECESSARY FILES
import os
import sys

data_fitting_py_files_dir = os.path.abspath("../../py_files/")
sys.path.insert(0, data_fitting_py_files_dir)

helper_func_py_files_dir = os.path.abspath("../../../../vampires_on_sky_calibration/programs/py_files/")
sys.path.insert(0, helper_func_py_files_dir)

import matplotlib.pyplot as plt
from jax import numpy as jnp
from scipy.optimize import minimize
from jax import grad, jit, lax
import numpy as np
import general_jax as general
import data_plotting_jax as data_plotting
import helper_functions_jax as funcs
import reading_data
import instrument_matrices_jax_troubleshooting as instrument_matrices

## LOADING DATA

HWP_angs = jnp.array([0., 11.25, 22.5, 33.75, 45., 56.25, 67.5, 78.75])
IMR_angs = jnp.array([45., 57.5, 70., 82.5, 95., 107.5, 120., 132.5])
wavelengths = jnp.array([625, 675, 725, 750, 775])
angles = [HWP_angs, IMR_angs]

# Last index is for choosing the wavelength
wavelength_index = 3
wavelength = 750

double_differences = jnp.array(np.load("../../../data/20230914/double_diffs_20230914.npy")[0:8, 0:8, wavelength_index])
double_sums = jnp.array(np.load("../../../data/20230914/double_sums_20230914.npy")[0:8, 0:8, wavelength_index])
double_difference_errs = jnp.array(np.load("../../../data/20230914/double_diff_stds_20230914.npy")[0:8, 0:8, wavelength_index])
double_sum_errs = jnp.array(np.load("../../../data/20230914/double_sum_stds_20230914.npy")[0:8, 0:8, wavelength_index])
data = jnp.array([double_differences, double_sums])
stds = jnp.array([double_difference_errs, double_sum_errs])

reshaped_data = general.reshape_and_flatten(data)
reshaped_stds = general.reshape_and_flatten(stds)

## BOUNDS AND INITIAL GUESS

# NOTE: Customized angle bounds for each parameter
IMR_angle_bound = 1
LP_angle_bound = 5
HWP_angle_bound = 5
FLC_angle_bound = (-5, 5)

bounds = ((0, 1),  # EM Gain Ratio
          (-1 * LP_angle_bound, LP_angle_bound),  # linear polarizer angle (degrees)
          (-1, 1),  # delta_FLC (waves)
          (-1, 1),  # delta_opt (waves)
          (-1, 1),  # delta_derot (waves)
          (0, 0.499999),  # delta_HWP (waves)
          FLC_angle_bound,  # rot_FLC (degrees)
          (-90, 90),  # rot_opt (degrees)
          (-1 * IMR_angle_bound, IMR_angle_bound),  # offset_derot (degrees)
          (-1 * HWP_angle_bound, HWP_angle_bound),  # offset_HWP (degrees)
          (0, 1))  # epsilon_opt (unitless)

# Based on values in Six_And_Eleven_Parameter_Model_Best_MCMC_and_Scipy_Fits - having zeros for angles that're too large
theta_pol = 0
delta_m3 = 0  # (waves) - assumed to be a perfect mirror for now
epsilon_m3 = 0  # Using the M3 diattenuation from :all_unpolarized_standards_matrix_inversion_m3_diatttenuation"
offset_m3 = 0  # NOTE: Made this zero too for testing purposes
delta_HWP = 0.451  # Add your actual delta_HWP value
offset_HWP = -2.642  # Add your actual offset_HWP value
delta_derot = 0.32  # Add your actual delta_derot value
offset_derot = 0  # Add your actual offset_derot value
delta_opts = 0  # Add your actual delta_opts value
epsilon_opts = 0  # Add your actual epsilon_opts value
rot_opts = 0  # Add your actual rot_opts value
delta_FLC = 0.5  # Add your actual delta_FLC value
rot_FLC = 0  # Add your actual rot_FLC value
em_gain = 1.2  # Add your actual em_gain value

## MINIMIZATION AND PLOTTING

import numpy as np
from scipy.optimize import minimize

# Small value to remove error bars
log_f = -10

# Defining model angles
model_angles = np.linspace(0, 90, 100)

# List to store all the solutions 
solns = []

# Initial guess based on the parameters you want to minimize
initial_guess = jnp.array([theta_pol, delta_HWP, offset_HWP, delta_derot, offset_derot,
                           delta_opts, epsilon_opts, rot_opts, delta_FLC, rot_FLC, em_gain])

all_params = [delta_m3, epsilon_m3, offset_m3, delta_HWP, offset_HWP,
              delta_derot, offset_derot, delta_opts, epsilon_opts, rot_opts, delta_FLC,
              rot_FLC, em_gain]

# Fixed parameters not included in the fitting process
fixed_params = {'delta_m3': delta_m3, 'epsilon_m3': epsilon_m3, 'offset_m3': offset_m3, 'parang': 0, 'altitude': 0}

# Define the bounds for the parameters (including em_gain)
bounds = [
    (-5, 5),  # theta_pol
    (0, 0.5),  # delta_HWP
    (-5, 5),  # offset_HWP
    (0, 0.5),  # delta_derot
    (-5, 5),  # offset_derot
    (-0.5, 0.5),  # delta_opts
    (0, 0.1),  # epsilon_opts
    (-90, 90),  # rot_opts
    (0.4, 0.6),  # delta_FLC
    (-90, 90),  # rot_FLC
    (0, 2)  # em_gain
]

# Defining the negative log-likelihood function with JAX
@jit
def nll(params, model, HWP_angs, IMR_angs, data, stds):
    theta_pol, delta_HWP, offset_HWP, delta_derot, offset_derot, delta_opts, epsilon_opts, rot_opts, delta_FLC, rot_FLC, em_gain = params
    all_params = {'delta_m3': fixed_params['delta_m3'], 'epsilon_m3': fixed_params['epsilon_m3'], 'offset_m3': fixed_params['offset_m3'], 
                  'delta_HWP': delta_HWP, 'offset_HWP': offset_HWP, 'delta_derot': delta_derot, 'offset_derot': offset_derot, 
                  'delta_opts': delta_opts, 'epsilon_opts': epsilon_opts, 'rot_opts': rot_opts, 'delta_FLC': delta_FLC, 
                  'rot_FLC': rot_FLC, 'em_gain': em_gain, 'parang': fixed_params['parang'], 'altitude': fixed_params['altitude']}
    this_model = -instrument_matrices.internal_calibration_mueller_matrix(theta_pol, model, all_params, HWP_angs, IMR_angs)
    residuals = this_model - data

    likelihood = jnp.sum((residuals / stds) ** 2)
    return likelihood

# Initialize variables for the iterative minimization process
counter = 0
initial_likelihood = 100
post_likelihood = 90

# Starting off with the initial guess
model = instrument_matrices.full_system_mueller_matrix
initial_model = -instrument_matrices.internal_calibration_mueller_matrix(theta_pol, model, all_params, HWP_angs, IMR_angs)

# Convert to NumPy array for compatibility with scipy.optimize.minimize
initial_guess_np = np.array(initial_guess)

while post_likelihood < initial_likelihood * 0.99:
    counter += 1

    initial_likelihood = post_likelihood

    # Calculate the initial model and residuals
    initial_model = -instrument_matrices.internal_calibration_mueller_matrix(theta_pol, model, all_params, HWP_angs, IMR_angs)
    initial_residuals = initial_model - reshaped_data

    initial_likelihood = jnp.sum((initial_residuals / reshaped_stds) ** 2)

    print("Initial Likelihood: " + str(initial_likelihood))

    # Minimize the negative log-likelihood
    minimize_args = (model, HWP_angs, IMR_angs, reshaped_data, reshaped_stds)
    soln = minimize(nll, initial_guess_np, args=minimize_args, bounds=bounds, method="Nelder-Mead")

    # Save the solution
    solns.append(soln)

    # Recalculate the likelihood with the new solution
    theta_pol, delta_HWP, offset_HWP, delta_derot, offset_derot, delta_opts, epsilon_opts, rot_opts, delta_FLC, rot_FLC, em_gain = soln.x
    all_params = {'delta_m3': fixed_params['delta_m3'], 'epsilon_m3': fixed_params['epsilon_m3'], 'offset_m3': fixed_params['offset_m3'], 
                  'delta_HWP': delta_HWP, 'offset_HWP': offset_HWP, 'delta_derot': delta_derot, 'offset_derot': offset_derot, 
                  'delta_opts': delta_opts, 'epsilon_opts': epsilon_opts, 'rot_opts': rot_opts, 'delta_FLC': delta_FLC, 
                  'rot_FLC': rot_FLC, 'em_gain': em_gain, 'parang': fixed_params['parang'], 'altitude': fixed_params['altitude']}

    # Recalculate the likelihood with the new solution
    post_likelihood = jnp.sum((-instrument_matrices.internal_calibration_mueller_matrix(theta_pol, model, all_params, HWP_angs, IMR_angs) - reshaped_data) / reshaped_stds ** 2)

    print("Iteration #" + str(counter) + ": " + str(post_likelihood))

    print("MAXIMUM LIKELIHOOD ESTIMATES")
    print("")
    print("theta_pol (degrees): " + str(theta_pol))
    print("delta_HWP (waves): " + str(delta_HWP))
    print("offset_HWP (degrees): " + str(offset_HWP))
    print("delta_derot (waves): " + str(delta_derot))
    print("offset_derot (degrees): " + str(offset_derot))
    print("delta_opts (waves): " + str(delta_opts))
    print("epsilon_opts: " + str(epsilon_opts))
    print("rot_opts (degrees): " + str(rot_opts))
    print("delta_FLC (waves): " + str(delta_FLC))
    print("rot_FLC (degrees): " + str(rot_FLC))
    print("em_gain: " + str(em_gain))

    reshaped_data = general.reshape_and_flatten(data)
    reshaped_stds = general.reshape_and_flatten(stds)

    model_1 = -instrument_matrices.internal_calibration_mueller_matrix(theta_pol, model, all_params, HWP_angs, IMR_angs)
    residuals_1 = model_1 - reshaped_data

    data_plotting.plot_single_model_and_residuals(angles, angles, model_1, data,
                                                  residuals_1, stds, log_f, wavelength, fig_dimensions=(30, 20))

    # Reset initial guess
    initial_guess = soln.x

    print("Post Likelihood: " + str(post_likelihood))

print("End of Optimization")


TypeError: Cannot interpret value of type <class 'jaxlib.xla_extension.PjitFunction'> as an abstract array; it does not have a dtype attribute

In [2]:
## IMPORTING NECESSARY FILES
import os
import sys

data_fitting_py_files_dir = os.path.abspath("../../py_files/")
sys.path.insert(0, data_fitting_py_files_dir)

helper_func_py_files_dir = os.path.abspath("../../../../vampires_on_sky_calibration/programs/py_files/")
sys.path.insert(0, helper_func_py_files_dir)

import matplotlib.pyplot as plt
from jax import numpy as jnp
from scipy.optimize import minimize
from jax import grad, jit, lax
import numpy as np
import general_jax as general
import data_plotting_jax as data_plotting
import helper_functions_jax as funcs
import reading_data
import instrument_matrices_jax_troubleshooting as instrument_matrices

# Define test parameters
theta_pol = 0
model_params = {
    'delta_HWP': 0,
    'offset_HWP': 0,
    'delta_derot': 0.451,
    'offset_derot': -2.642,
    'delta_opts': 0.32,
    'epsilon_opts': 0,
    'rot_opts': 0,
    'delta_FLC': 0.5,
    'rot_FLC': 0,
}
fixed_params = {
    'delta_m3': 0,
    'epsilon_m3': 0,
    'offset_m3': 0,
    'em_gain': 1,
    'parang': 0,
    'altitude': 0,
}

HWP_angs = jnp.array([0., 11.25, 22.5, 33.75, 45., 56.25, 67.5, 78.75])
IMR_angs = jnp.array([45., 57.5, 70., 82.5, 95., 107.5, 120., 132.5])
wollaston_properties = instrument_matrices.get_wollaston_properties(cam_num=1)

# Convert beam property to appropriate numeric value outside of JAX-traced function
wollaston_properties['beam'] = 0  # Ensure beam is numeric

# Print expected inputs
print(f"Fixed Params: {fixed_params}")
print(f"Model Params: {model_params}")
print(f"HWP Angles: {HWP_angs}")
print(f"IMR Angles: {IMR_angs}")


Fixed Params: {'delta_m3': 0, 'epsilon_m3': 0, 'offset_m3': 0, 'em_gain': 1, 'parang': 0, 'altitude': 0}
Model Params: {'delta_HWP': 0, 'offset_HWP': 0, 'delta_derot': 0.451, 'offset_derot': -2.642, 'delta_opts': 0.32, 'epsilon_opts': 0, 'rot_opts': 0, 'delta_FLC': 0.5, 'rot_FLC': 0}
HWP Angles: [ 0.   11.25 22.5  33.75 45.   56.25 67.5  78.75]
IMR Angles: [ 45.   57.5  70.   82.5  95.  107.5 120.  132.5]
